# Network Prediction
This project predicts the salary, position, and future connections of the employees of a company using their logs of email exchanges. The raw data is stored in `email_prediction.txt`, a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people. The network also contains the node attributes `Department` and `ManagementSalary`. `Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [8]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Salary Prediction

Ddentify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

Create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.

The predictions is given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric is the Area Under the ROC Curve (AUC).

In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
def salary_predictions():
    
    df = pd.DataFrame(index=G.nodes())
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    
    unknown = df[df['ManagementSalary'].isnull()]
    features = ['clustering', 'degree', 'degree_centrality', 'closeness', 'betweeness', 'pr' ]
    X = unknown[features]
    train = df[~df['ManagementSalary'].isnull()]
    X_train = train[features]
    y_train = train['ManagementSalary']

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    X_scaled = scaler.transform(X)
    clf = LogisticRegression().fit(X_train_scaled, y_train)
    probs = clf.predict_proba(X_scaled)[:,1]
    result = pd.Series(probs, index=X.index) 
    return result
salary_predictions()

1       0.182372
2       0.460167
5       0.906187
8       0.148184
14      0.325667
18      0.215190
27      0.255241
30      0.278608
31      0.185124
34      0.139532
37      0.138974
40      0.225018
45      0.183901
54      0.209683
55      0.192732
60      0.208589
62      0.959168
65      0.461453
77      0.108798
79      0.159558
97      0.118974
101     0.109317
103     0.204844
108     0.159094
113     0.378859
122     0.101819
141     0.362022
142     0.716541
144     0.092002
145     0.311267
          ...   
913     0.095571
914     0.107660
915     0.034973
918     0.134612
923     0.074344
926     0.100134
931     0.091867
934     0.054784
939     0.052165
944     0.035623
945     0.072957
947     0.109813
950     0.150844
951     0.079107
953     0.077132
959     0.036788
962     0.037156
963     0.159761
968     0.093080
969     0.088206
974     0.089886
984     0.052953
987     0.100618
989     0.093384
991     0.093120
992     0.040781
994     0.037035
996     0.0357

### New Connections Prediction

Predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [87]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head()

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

Create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.

The predictions are given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

In [89]:
def new_connections_predictions():
    
    future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbors'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    
    unknown = future_connections[future_connections['Future Connection'].isnull()]
    features = ['preferential attachment', 'Common Neighbors']
    X = unknown[features]
    train = future_connections[~future_connections['Future Connection'].isnull()]
    X_train = train[features]
    y_train = train['Future Connection']

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    X_scaled = scaler.transform(X)
    clf = LogisticRegression().fit(X_train_scaled, y_train)
    probs = clf.predict_proba(X_scaled)[:,1]
    result = pd.Series(probs, index=X.index) 
    return result
new_connections_predictions()


(107, 348)    0.042438
(542, 751)    0.018056
(20, 426)     0.585127
(50, 989)     0.018171
(942, 986)    0.018296
(324, 857)    0.018155
(13, 710)     0.183715
(19, 271)     0.160734
(319, 878)    0.018211
(659, 707)    0.018068
(49, 843)     0.018247
(208, 893)    0.018095
(377, 469)    0.013681
(405, 999)    0.028930
(129, 740)    0.027301
(292, 618)    0.027559
(239, 689)    0.018219
(359, 373)    0.015409
(53, 523)     0.043006
(276, 984)    0.018241
(202, 997)    0.018265
(604, 619)    0.046334
(270, 911)    0.018217
(261, 481)    0.090268
(200, 450)    0.735997
(213, 634)    0.018020
(644, 735)    0.046062
(346, 553)    0.017733
(521, 738)    0.017183
(422, 953)    0.027112
                ...   
(672, 848)    0.018217
(28, 127)     0.919928
(202, 661)    0.017684
(54, 195)     0.998994
(295, 864)    0.018137
(814, 936)    0.018038
(839, 874)    0.018296
(139, 843)    0.018147
(461, 544)    0.016801
(68, 487)     0.016734
(622, 932)    0.018109
(504, 936)    0.025762
(479, 528) 